In [8]:
# !ollama pull gemma2:9b
# !ollama pull nextfire/paraphrase-multilingual-minilm:l12-v2


In [11]:
from scripts.ollama_client import OllamaClient
from scripts.text_indexer import TextIndexer

In [ ]:
from scripts.ollama_client import OllamaClient
from scripts.text_indexer import TextIndexer
from qdrant_client import QdrantClient
from qdrant_client.http import models

ollama_client = OllamaClient(
    host="localhost",
    llm_port=11434,
    temperature = 0.0,
    model_name="gemma2:9b",
    embedding_model="nextfire/paraphrase-multilingual-minilm:l12-v2"
)

# Initialize QdrantClient
qdrant_client = QdrantClient(host="localhost", port=6333)

# Sample document
text = "The quick brown fox jumps over the lazy dog."

# Create an indexer
indexer = TextIndexer(text=text, ollama_client=ollama_client, qdrant_client=qdrant_client)

# Process and index the document
indexer.process_file()

# User query
query = "What is the color of the fox?"

# Generate query embedding
query_vector = ollama_client.get_embeddings([query])[0]

# Retrieve top K relevant chunks from Qdrant
top_k = 5
retrieved_chunks = indexer.retrieve(query_vector=query_vector, top_k=top_k)

# Combine relevant chunks with query and generate response using Ollama
relevant_text = "\n".join([chunk["payload"]["text"] for chunk in retrieved_chunks])
final_prompt = f"Given the following information:\n{relevant_text}\n\nAnswer the question: {query}"

response = ollama_client.generate(final_prompt)
print("Response:", response)


Response: The color of the fox is **brown**. 

This is stated directly in the sentence: "The quick brown fox..."  

